## problem statement

Stakeholder: City planner / City

Problem to solve: What factors contribute to traffic incidents in Chicago?

What will our model produce?: Our model will determine which primary contributors are most relevant in a car accident in Chicago

#EDA

In [1]:
import pandas as pd

In [2]:
crash_df = pd.read_csv('data/Traffic_Crashes_-_crashes.csv')

In [3]:
people_df = pd.read_csv('data/Traffic_Crashes_-_people.csv', low_memory=False)

In [4]:
vehicle_df = pd.read_csv('data/Traffic_Crashes_-_vehicles.csv', low_memory=False)

## Crash_df EDA

In [12]:
crash_df.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,JD292400,NaN,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,JD293602,NaN,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,JD290451,NaN,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN


In [11]:
boolean = crash_df.duplicated(subset=['CRASH_RECORD_ID']).any()
boolean

False

In [6]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554228 entries, 0 to 554227
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                554228 non-null  object 
 1   RD_NO                          549766 non-null  object 
 2   CRASH_DATE_EST_I               41950 non-null   object 
 3   CRASH_DATE                     554228 non-null  object 
 4   POSTED_SPEED_LIMIT             554228 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         554228 non-null  object 
 6   DEVICE_CONDITION               554228 non-null  object 
 7   WEATHER_CONDITION              554228 non-null  object 
 8   LIGHTING_CONDITION             554228 non-null  object 
 9   FIRST_CRASH_TYPE               554228 non-null  object 
 10  TRAFFICWAY_TYPE                554228 non-null  object 
 11  LANE_CNT                       198970 non-null  float64
 12  ALIGNMENT                     

In [16]:
len(crash_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts())

40

### Columns to Drop for crash_df: 
* RD_NO - Plice Dep. Report number, not relevant
* CRASH_DATE_EST_I - used when crash is reported to police days after the crash, lots of NAN for this column 
* CRASH_DATE - this dataframe inclues crash day of week, hour and month so we can drop the specific date. 
* REPORT_TYPE - administrative report type, not relevant to finding out cause of crash
* HIT_AND_RUN_I - not relevant to finding out cause of crash
* DATE_POLICE_NOTIFIED - not relevant to finding out cause of crash
* STREET_NO - probably could just keep `STREET_NAME`
* STREET_DIRECTION
* BEAT_OF_OCCURENCE - relevant to the chicaco police boundaries
* PHOTOS_TAKEN_I 
* STATEMENTS_TAKEN

### Up for debate:
* PRIM_CONTRIBUTORY_CAUSE - hmm,  207833 "unable to determine", there are a total of 40 primary causes listed. it would be a lot to sift through
* Sec_CONTRIBUTORY_CAUSE - leaning towards no
Injuries - I think we should just keep `INJURIES_TOTAL` and drop the details: 
* MOST_SEVERE_INJURY - 
* INJURIES_FATAL - same as above
* INJURIES_INCAPACITATING same as above
* INJURIES_NON_INCAPACITATING
* INJURIES_REPORTED_NOT_EVIDENT
* INJURIES_NO_INDICATION
* INJURIES_UNKNOWN


## dropping unnecessary rows from `crash_df`

In [49]:
crash_df_dropped = crash_df.drop(columns=["CRASH_RECORD_ID", "RD_NO", "CRASH_DATE_EST_I", 
                                   "CRASH_DATE", "REPORT_TYPE", "HIT_AND_RUN_I", 
                                   "DATE_POLICE_NOTIFIED", "STREET_DIRECTION", "BEAT_OF_OCCURRENCE", 
                                   "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", "SEC_CONTRIBUTORY_CAUSE", 
                                   "MOST_SEVERE_INJURY", "INJURIES_FATAL", "INJURIES_INCAPACITATING", 
                                   "INJURIES_NON_INCAPACITATING", "INJURIES_REPORTED_NOT_EVIDENT", 
                                   "INJURIES_NO_INDICATION", "INJURIES_UNKNOWN", "STREET_NO"])
crash_df_dropped.head()



,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,...,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,INJURIES_TOTAL,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,ONE-WAY,NaN,STRAIGHT AND LEVEL,DRY,...,NaN,NaN,2.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,4.0,STRAIGHT AND LEVEL,DRY,...,NaN,NaN,2.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,NaN,STRAIGHT AND LEVEL,DRY,...,NaN,NaN,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),NaN,STRAIGHT AND LEVEL,DRY,...,NaN,NaN,2.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DRIVEWAY,NaN,STRAIGHT AND LEVEL,DRY,...,NaN,NaN,2.0,0.0,14,4,7,NaN,NaN,NaN


## Looking at `people_df`

In [37]:
people_df.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [38]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224613 entries, 0 to 1224612
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1224613 non-null  object 
 1   PERSON_TYPE            1224613 non-null  object 
 2   CRASH_RECORD_ID        1224613 non-null  object 
 3   RD_NO                  1214849 non-null  object 
 4   VEHICLE_ID             1200346 non-null  float64
 5   CRASH_DATE             1224613 non-null  object 
 6   SEAT_NO                250075 non-null   float64
 7   CITY                   900162 non-null   object 
 8   STATE                  910974 non-null   object 
 9   ZIPCODE                821859 non-null   object 
 10  SEX                    1206115 non-null  object 
 11  AGE                    872486 non-null   float64
 12  DRIVERS_LICENSE_STATE  722228 non-null   object 
 13  DRIVERS_LICENSE_CLASS  621288 non-null   object 
 14  SAFETY_EQUIPMENT  

In [39]:
boolean = people_df.duplicated(subset=['CRASH_RECORD_ID']).any()
boolean

True

### Columns to Drop for people_df: 

NOTE: Looks like there are duplicates of `CRASH_RECORD_ID` - we will need to figure out how to handle that before joining. 

* PERSON_ID 
* PERSON_TYPE
* RD_NO *we could potentially use this for joining but CRASH_RECORD_ID works too
* VEHICLE_ID
* CRASH_DATE
* SEAT_NO
* CITY *Keep State I thnk it would be interesting to see if out of state drivers are a factor*
* ZIPCODE
* SEX
* AGE
* DRIVER_LICENSE_STATE *there's more non-null with `STATE`*
* DRIVER_LICENSE_CLASS
* SAFETY_EQUIPMENT
* EJECTION
* INJURY CLASSIFICATION
* HOSPITAL
* EMS_AGENCY
* EMS_RUN_NO
* DRIVER_VISION
* PHYSICAL_CONDITION
* ... *probably easier to note the ones we want to keep instead of drop*

### KEEP: 
* CRASH_RECORD_ID
* STATE
* SAFETY_EQUIPMENT
* DRIVER_ACTION
* PEDPEDAL_VISIBILITY
* PEDPEDAL_LOCATION
* BAC_RESULT
* BAC_RESULT_VALUE
* CELL_PHONE_USE


In [52]:
people_df_dropped = crash_df.loc[:, ["CRASH_RECORD_ID", "STATE", "SAFETY_EQUIPMENT", "DRIVER_ACTION", 
                                      "PEDPEDAL_VISIBILITY", "PEDPEDAL_LOCATION", "BAC_RESULT", 
                                      "BAC_RESULT_VALUE", "CELL_PHONE_USE"]]
people_df_dropped.head()

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['STATE', 'SAFETY_EQUIPMENT', 'DRIVER_ACTION', 'PEDPEDAL_VISIBILITY',\n       'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT_VALUE',\n       'CELL_PHONE_USE'],\n      dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

## Looking at `vehicles_df`

In [41]:
vehicle_df.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,04/13/2020 10:50:00 PM,2,DRIVER,NaN,827212.0,NaN,BUICK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1134612 entries, 0 to 1134611
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1134612 non-null  int64  
 1   CRASH_RECORD_ID           1134612 non-null  object 
 2   RD_NO                     1125395 non-null  object 
 3   CRASH_DATE                1134612 non-null  object 
 4   UNIT_NO                   1134612 non-null  int64  
 5   UNIT_TYPE                 1132985 non-null  object 
 6   NUM_PASSENGERS            169810 non-null   float64
 7   VEHICLE_ID                1108583 non-null  float64
 8   CMRC_VEH_I                20992 non-null    object 
 9   MAKE                      1108578 non-null  object 
 10  MODEL                     1108435 non-null  object 
 11  LIC_PLATE_STATE           1011371 non-null  object 
 12  VEHICLE_YEAR              928020 non-null   float64
 13  VEHICLE_DEFECT            1

In [43]:
boolean = vehicle_df.duplicated(subset=['CRASH_RECORD_ID']).any()
boolean

True

### Columns to Drop for vehicle_df: 

NOTE: Looks like there are duplicates of `CRASH_RECORD_ID` - we will need to figure out how to handle that before joining. 

*probably easier to note the ones we want to keep instead of drop*

### KEEP: 
* CRASH_RECORD_ID
* VEHICLE YEAR
* MANEUVER
* EXCEED_SPEED_LIMIT_I
*  ... ? 

*Do we even use this data?* 

## join data frames

## eda